In [11]:
import pandas as pd

df_tags = pd.read_csv('../data/raw/ml-25m/tags.csv')
df_gscores = pd.read_csv('../data/raw/ml-25m/genome-scores.csv')
df_gtags = pd.read_csv('../data/raw/ml-25m/genome-tags.csv')
df_links = pd.read_csv('../data/raw/ml-25m/links.csv')
df_movies = pd.read_csv('../data/raw/ml-25m/movies.csv')
df_ratings = pd.read_csv('../data/raw/ml-25m/ratings.csv')

In [12]:
##create large dataframe
#The same movieId refers to the same movie between `ratings.csv`, `tags.csv`, `movies.csv`, and `links.csv`
df = pd.merge(df_movies, df_tags, on=['movieId'], how='left')
df = pd.merge(df, df_ratings, on=['movieId', 'userId'], how='left')

df.head()

,movieId,title,genres,userId,tag,timestamp_x,rating,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,791.0,Owned,1.515175e+09,4.5,1.515175e+09
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1048.0,imdb top 250,1.172144e+09,NaN,NaN
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1361.0,Pixar,1.216146e+09,2.5,1.216146e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3164.0,Pixar,1.223305e+09,5.0,1.223264e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3164.0,time travel,1.223305e+09,5.0,1.223264e+09


In [13]:
# Drop rows where 'rating' and 'tag is NaN
df = df.dropna(subset=['rating', 'tag'])

# Drop 'timestamp_x' and 'timestamp_y' columns
df.drop('timestamp_x', axis=1, inplace=True)
df.drop('timestamp_y', axis=1, inplace=True)


In [14]:
# Combine rows with the same 'movieId' but distinct tags.
tags = df.groupby('movieId')['tag'].apply(lambda x: '|'.join(x.unique())).reset_index()
df = pd.merge(df, tags, on='movieId', how='left')

# Clean up the tags
df.drop('tag_x', axis=1, inplace=True)
df.rename(columns={'tag_y': 'tags'}, inplace=True)

# Safe new dataframe without one-Hot encoding
df.to_csv('preprocessed_data_no_one_hot_encoding.csv') 

In [5]:
# Use one-hot encoding to transform the genres and into a format suitable for machine learning model

genres_dummies = df['genres'].str.get_dummies(sep='|')
df = pd.concat([df, genres_dummies], axis=1)
df.drop('genres', axis=1, inplace=True)
df.drop('(no genres listed)', axis=1, inplace=True)


In [6]:
# Splitting up the tags with one-hot encoding

# Determine the frequency of each tag
tag_counts = df['tags'].str.split('|').explode().value_counts()

# Filter tags that occur more than a certain threshold, for example, 18000 times
frequent_tags = tag_counts[tag_counts > 18000].index.tolist()

# Keep only the frequent tags in your DataFrame
df['filtered_tags'] = df['tags'].apply(lambda x: '|'.join([tag for tag in x.split('|') if tag in frequent_tags]))

# Now perform one-hot encoding on the filtered tags
tags_dummies = df['filtered_tags'].str.get_dummies(sep='|')
df = pd.concat([df, tags_dummies], axis=1)


In [7]:
# Transform float values in column 'userId' into integer
df['userId'] = df['userId'].astype(int)

# Drop last useless columns
df.drop('tags', axis=1, inplace=True)
df.drop('Owned', axis=1, inplace=True)

In [8]:
df.head()

,movieId,title,userId,rating,Action,Adventure,Animation,Children,Comedy,Crime,...,visually appealing,visually stunning,want,want it,want to see again,war,watch the credits,weird,whimsical,witty
0,1,Toy Story (1995),791,4.5,0,1,1,1,1,0,...,0,0,0,0,1,0,0,0,0,1
1,1,Toy Story (1995),1361,2.5,0,1,1,1,1,0,...,0,0,0,0,1,0,0,0,0,1
2,1,Toy Story (1995),3164,5.0,0,1,1,1,1,0,...,0,0,0,0,1,0,0,0,0,1
3,1,Toy Story (1995),3164,5.0,0,1,1,1,1,0,...,0,0,0,0,1,0,0,0,0,1
4,1,Toy Story (1995),3448,3.0,0,1,1,1,1,0,...,0,0,0,0,1,0,0,0,0,1


In [9]:
# Safe new dataframe
df.to_csv('preprocessed_data.csv') 

##### Deadlines:
- Report 1. Data viz (5 graphs) + preprocessing. Friday, May 17
- Report 2. Modelling. Friday, June 14
- Final report. Friday, June 21
